In [1]:
# Imports and setup
import pandas as pd

from pathlib import Path

root = Path.cwd().parent

test_df = pd.read_csv(root/'data'/'test.csv')
store_df = pd.read_csv(root/'data'/'store.csv')
df = test_df.merge(store_df, how='left', on='Store')


In [2]:
# Check if CompetitionOpenSinceMonth and Year are always missing together

month_missing = df['CompetitionOpenSinceMonth'].isnull()
year_missing = df['CompetitionOpenSinceYear'].isnull()

# Compare both masks
inconsistencies = df[month_missing != year_missing]

print(f"Number of inconsistent rows: {len(inconsistencies)}")
if len(inconsistencies) > 0:
    display(inconsistencies.head())
else:
    print("✅ Month and Year are always missing together.")

Number of inconsistent rows: 0
✅ Month and Year are always missing together.


In [3]:
# Check if missingness of Promo2SinceWeek, Promo2SinceYear, and PromoInterval perfectly matches Promo2 == 0

all_missing = (
    df['Promo2SinceWeek'].isnull() & 
    df['Promo2SinceYear'].isnull() & 
    df['PromoInterval'].isnull()
)

mismatch = df[all_missing != (df['Promo2'] == 0)]

print(f"Number of rows where missingness doesn't match Promo2 == 0: {len(mismatch)}")
if len(mismatch) > 0:
    display(mismatch.head())
else:
    print("✅ Missing Promo2SinceWeek, Promo2SinceYear, and PromoInterval perfectly match Promo2 == 0.")
 

Number of rows where missingness doesn't match Promo2 == 0: 12


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
1,29106,3,5,8/14/2015,1,0,0,0,a,a,18956,12,2014,1,NaN,NaN,NaN
7,29962,3,4,8/13/2015,1,0,0,0,a,a,18956,12,2014,1,NaN,NaN,NaN
13,30818,3,3,8/12/2015,1,0,0,0,a,a,18956,12,2014,1,NaN,NaN,NaN
19,31674,3,2,8/11/2015,1,1,0,1,a,a,18956,12,2014,1,NaN,NaN,NaN
25,32530,3,1,8/10/2015,1,1,0,1,a,a,18956,12,2014,1,NaN,NaN,NaN


In [4]:
### Preliminary Feature Engineering

## Extract date features

# Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Break down into year, month, and day of the week
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.dayofweek

In [5]:
# Handle missing values in Rossmann dataset

# Create a flag for missing values and fill missing CompetitionDistance with median
df['CompetitionDistanceMissing'] = df['CompetitionDistance'].isnull().astype(int)
df['CompetitionDistance'] = df['CompetitionDistance'].fillna(df['CompetitionDistance'].median())

# Flag missing competition open dates and fill missing values with the current row's date (assume competition just opened)
df['CompetitionOpenMissing'] = df['CompetitionOpenSinceYear'].isnull().astype(int)
df['CompetitionOpenSinceMonth'] = df['CompetitionOpenSinceMonth'].fillna(df['Date'].dt.month)
df['CompetitionOpenSinceYear'] = df['CompetitionOpenSinceYear'].fillna(df['Date'].dt.year)

# Cast to int (original columns are float due to NaNs)
df['CompetitionOpenSinceMonth'] = df['CompetitionOpenSinceMonth'].astype(int)
df['CompetitionOpenSinceYear'] = df['CompetitionOpenSinceYear'].astype(int)

# Fill missing promo-related values for stores not participating in Promo2
df['Promo2SinceYear'] = df['Promo2SinceYear'].fillna(0).astype(int)
df['Promo2SinceWeek'] = df['Promo2SinceWeek'].fillna(0).astype(int)
df['PromoInterval'] = df['PromoInterval'].fillna('None')

# Fill missing 'Open' values with 1 (assume store is open)
df['Open'] = df['Open'].fillna(1).astype(int)

# Confirm all missing values are handled
missing = df.isnull().sum()
missing = missing[missing > 0]

if missing.empty:
    print("Remaining missing values: 0")
else:
    print("Remaining missing values:\n")
    print(missing)


Remaining missing values: 0


🔧 FEATURE ENGINEERING:

In [6]:
## Compute months since competition opened

# Create datetime for competition opening
df['CompetitionOpenSince'] = pd.to_datetime(
    {
        'year': df['CompetitionOpenSinceYear'],
        'month': df['CompetitionOpenSinceMonth'],
        'day': 1
    }
)

# Calculate difference in months between 'Date' and 'CompetitionOpenSince'
df['MonthsSinceCompOpened'] = (
    (df['Date'].dt.year - df['CompetitionOpenSince'].dt.year) * 12 +
    (df['Date'].dt.month - df['CompetitionOpenSince'].dt.month)
).clip(lower=0).fillna(0).astype(int)

In [7]:
##  Define IsPromoIntervalActive (1 if promo active during this month, else 0)

# Mapping month names to numbers
month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
    'Sept': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Function to determine if the promo is active in the current month
def promo_active(row):
    if pd.isna(row['PromoInterval']) or row['PromoInterval'] in ['None', '']:
        return 0
    promo_months = [month_map[month] for month in row['PromoInterval'].split(',')]
    return int(row['Date'].month in promo_months)

# Apply function row-wise
df['IsPromoIntervalActive'] = df.apply(promo_active, axis=1)


In [8]:
## Additional date features

# Week of the year (1–52)
df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)

# Days since start of dataset (continuous time progression)
df['DaysSinceStart'] = (df['Date'] - df['Date'].min()).dt.days


In [9]:
## Label Encoding for Categorical Features

from sklearn.preprocessing import LabelEncoder

categorical_cols = ['StateHoliday', 'StoreType', 'Assortment']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Cast all values to string
    label_encoders[col] = le

print("✅ Label encoding applied (with type-safe conversion) to:", categorical_cols)


✅ Label encoding applied (with type-safe conversion) to: ['StateHoliday', 'StoreType', 'Assortment']


In [10]:
# Drop columns not needed for modeling
df = df.drop(columns=[
    'Date',
    'CompetitionOpenSince',
    'CompetitionOpenSinceMonth',
    'CompetitionOpenSinceYear',
    'PromoInterval'
    ]
)


In [11]:
# Rearrange columns
feature_list = [
    'Id',
    'Store',
    'DayOfWeek',
    'Open',
    'Promo',
    'StateHoliday',
    'SchoolHoliday',
    'StoreType',
    'Assortment',
    'CompetitionDistance',
    'Promo2',
    'Promo2SinceWeek',
    'Promo2SinceYear',
    'Year',
    'Month',
    'CompetitionDistanceMissing',
    'CompetitionOpenMissing',
    'MonthsSinceCompOpened',
    'IsPromoIntervalActive',
    'WeekOfYear',
    'DaysSinceStart'
 ]

df = df[feature_list]

# Preview the updated DataFrame
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Id                          72 non-null     int64
 1   Store                       72 non-null     int64
 2   DayOfWeek                   72 non-null     int32
 3   Open                        72 non-null     int64
 4   Promo                       72 non-null     int64
 5   StateHoliday                72 non-null     int64
 6   SchoolHoliday               72 non-null     int64
 7   StoreType                   72 non-null     int64
 8   Assortment                  72 non-null     int64
 9   CompetitionDistance         72 non-null     int64
 10  Promo2                      72 non-null     int64
 11  Promo2SinceWeek             72 non-null     int64
 12  Promo2SinceYear             72 non-null     int64
 13  Year                        72 non-null     int32
 14  Month       

,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,Promo2SinceWeek,Promo2SinceYear,Year,Month,CompetitionDistanceMissing,CompetitionOpenMissing,MonthsSinceCompOpened,IsPromoIntervalActive,WeekOfYear,DaysSinceStart
0,29105,1,4,1,1,0,1,0,0,4950,...,0,0,2015,8,0,0,45,0,33,13
1,29106,3,4,1,0,0,0,0,0,18956,...,0,0,2015,8,0,0,8,0,33,13
2,29107,7,4,1,0,0,1,0,0,10077,...,14,2011,2015,8,0,0,71,0,33,13
3,29108,8,4,1,0,0,1,1,0,20315,...,0,0,2015,8,0,0,105,0,33,13
4,29109,9,4,1,1,0,0,0,1,27641,...,0,0,2015,8,0,0,0,0,33,13


# Prediction

In [12]:
import joblib
import numpy as np

# Load the trained model
best_model = joblib.load(root/'results'/'xgb_model.pkl')

X_test = df.drop(columns=['Id'])
y_pred = best_model.predict(X_test)
y_pred_truncated = np.maximum(y_pred, 0)

# Create a DataFrame with Id and predicted Sales
predictions_df = pd.DataFrame({
    'Id': df['Id'],
    'Sales': y_pred_truncated
})

# Save the predictions to a CSV file
predictions_df.to_csv(root/'results'/'predictions.csv', index=False)
